In [100]:
%config IPCompleter.greedy=True

In [101]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

In [102]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import scipy.io
import glob
from scipy.io import loadmat
import scipy
import pandas as pd
import io
import numpy as np
import cv2
import json
import os
import os.path
import cv2 as cv
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision import transforms, utils
import math
from skimage import io, transform
from collections import namedtuple
from enum import Enum
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from PIL import Image
from matplotlib import cm
import shutil
import codecs 
import timeit
import math
from tqdm import tqdm
import progressbar
%matplotlib inline
import time
import pylab as pl
from IPython import display
import gc
import subprocess

In [232]:
def capture(path):    
    cap = cv2.VideoCapture(path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    fc = 0
    ret = True
    while (fc < frameCount  and ret):
        ret, buf[fc] = cap.read()
        fc += 1
    cap.release()
    return buf    

In [238]:
def get_feature_vector(buf):
    feature_buf = []
    for i  in range(len(buf)):
      feature_buf.append(buf[i].mean(axis=0).mean(axis=0))
    return feature_buf

In [239]:
def get_key_frames(buf,feature_buf,threshold):
    key_frame = []
    key_frame.append(buf[0])
    i = 0
    while i < (len(buf)):
        for j in range(i+1,len(buf)):
            mse = np.mean((feature_buf[i] - feature_buf[j])**2)
            if mse > threshold:
                key_frame.append(buf[j])
                i = j
                break
        i = i+1   
    return key_frame   

In [228]:
def visualizer(video_array):    
    plt.ion()
    for i in range(len(video_array)):
        plt.imshow(cv2.cvtColor(video_array[i], cv2.COLOR_BGR2RGB))
        #plt.show()
        display.clear_output(wait=True)
        display.display(pl.gcf())

In [263]:
def is_similar(key_f_input,key_f_existing,
               threshold_match,threshold_compare): # threshold_match :: ratio threthold
                                                # threshold_compare :: feature compare threshold
    feature_buf_input = get_feature_vector(key_f_input)
    feature_buf_existing = get_feature_vector(key_f_existing)
    match_flag = False
    counter = 0
    for i in range(len(key_f_input)): # elli bn3ml 3leh retrieval
        for j in range(len(key_f_existing)): # wa7ed mn el database
            mse = np.mean((feature_buf_existing[i] - feature_buf_input[j])**2)
            if mse<threshold_compare:
                counter = counter + 1
                break
    ratio_input = counter/len(key_f_input) # ratio of input video key frames to counter
    ratio_existing = counter/len(key_f_existing) # ratio of existing video key frames to counter
    ratio_maximum = max(ratio_input,ratio_existing) # the highest ratio 
    if ratio_maximum >= threshold_match:
        match_flag = True
    return match_flag    

In [242]:
path0 = 'cbvrs3.mp4'
buf0 = capture(path0)
threshold_key_frame = 1
buf_feature0 =  get_feature_vector(buf0)
key_f0 = get_key_frames(buf0,buf_feature0,threshold_key_frame)

path1 = 'cbvrs4.mp4'
buf1 = capture(path1)
buf_feature1 =  get_feature_vector(buf1)
key_f1 = get_key_frames(buf1,buf_feature1,threshold_key_frame)

In [257]:
visualizer(key_f1);

In [264]:
is_similar(key_f1,key_f0,0.5,1)

1.0
0.3333333333333333


False